In [1]:
import json
import os
import copy
import numpy as np
import pandas as pd
from tqdm import tqdm
import time
import sys
sys.path.append("../")

from cfmining.algorithms import MAPOFCEM
from cfmining.predictors import GeneralClassifier_Shap
from cfmining.action_set import ActionSet
from cfmining.utils import get_data_model, DeepPipeExplainer
from cfmining.baselines import Bruteforce, MAPOCAM, Nice, Dice


%load_ext autoreload
%autoreload 2

In [2]:
SEED = 0

## Helper

In [3]:
def run_experiments(
        method,
        individuals, 
        model, 
        output_file = None,
    ):
    results = []

    if not output_file is None:
        folder = "/".join(output_file.split("/")[:-1])
        if not os.path.exists(folder):
            os.makedirs(folder, exist_ok = True)

    for i in tqdm(range(len(individuals))):
        individual = individuals.iloc[i]
        try:
            model.clear_cache()
        except:
            pass
        start = time.time()
        method.fit(individual.values)
        end = time.time()

        solutions = method.solutions
        
        results.append({
            "individual" : individual.values.tolist(),
            "prob" : model.predict_proba(individual.values),
            "time" : end - start,
            "n_solutions" : len(method.solutions),
            "solutions" : solutions,
        })

        print(f"Prob. max counter: {method.prob_max_counter} | Prob: {results[-1]['prob']:.2f}")
        if output_file is not None:
            pd.DataFrame(results).to_csv(output_file, index=False)

        

    results = pd.DataFrame(results)
    if output_file is not None:
        results.to_csv(output_file, index=False)
    else:
        return results

## German

In [4]:
X_train, Y_train, model, outlier_detection, individuals = get_data_model("german", "MLPClassifier")
individuals = individuals.sample(n = 50, random_state=SEED)

In [5]:
not_mutable_features = ['Age', 'OwnsHouse', 'isMale', 'JobClassIsSkilled', 'Single', 'ForeignWorker', 'RentsHouse']
mutable_features = [feat for feat in X_train.columns if feat not in not_mutable_features]
continuous_features = ['Age', 'LoanDuration', 'LoanAmount', 'LoanRateAsPercentOfIncome','YearsAtCurrentHome']
categoric_features = [col for col in X_train.columns if col not in continuous_features]

In [6]:
action_set = ActionSet(X = X_train, default_step_size = 0.05, mutable_features = mutable_features)
deep_pipe_explainer = DeepPipeExplainer(model, X_train.sample(100))
model_shap = GeneralClassifier_Shap(model, outlier_detection, X_train, shap_explainer="custom", threshold = 0.5, explainer = deep_pipe_explainer)
model_shap_simple = GeneralClassifier_Shap(model, outlier_detection, X_train)

### MAPOFCEM

In [7]:
method = MAPOFCEM(
    action_set = action_set,
    classifier = model_shap,
    compare = "percentile",
    max_changes = 3
)

run_experiments(
    method,
    individuals=individuals,
    model=model_shap,
    output_file=f"../results/mlp/german/mapofcem_percentile.csv"
)

  0%|                                                                                                                                   | 0/50 [00:00<?, ?it/s]

  2%|██▍                                                                                                                        | 1/50 [00:00<00:09,  4.90it/s]

Prob. max counter: 7 | Prob: 0.44


  4%|████▉                                                                                                                      | 2/50 [00:00<00:22,  2.09it/s]

Prob. max counter: 42 | Prob: 0.30


...

Prob. max counter: 60 | Prob: 0.14


 98%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▌  | 49/50 [00:18<00:00,  3.53it/s]

Prob. max counter: 12 | Prob: 0.39
Prob. max counter: 6 | Prob: 0.47


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 50/50 [00:19<00:00,  2.63it/s]

Prob. max counter: 7 | Prob: 0.40


In [8]:
method = MAPOFCEM(
    action_set = action_set,
    classifier = model_shap_simple,
    compare = "percentile",
    max_changes = 3
)

run_experiments(
    method,
    individuals=individuals,
    model=model_shap,
    output_file=f"../results/mlp/german/mapofcem_perm_percentile.csv"
)

  2%|██▍                                                                                                                        | 1/50 [00:01<01:25,  1.75s/it]

Prob. max counter: 7 | Prob: 0.44


  4%|████▉                                                                                                                      | 2/50 [00:02<01:07,  1.41s/it]

Prob. max counter: 33 | Prob: 0.30


  6%|███████▍                                                                                                                   | 3/50 [00:04<01:19,  1.68s/it]

...

Prob. max counter: 68 | Prob: 0.39


 98%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▌  | 49/50 [00:41<00:00,  1.63it/s]

Prob. max counter: 9 | Prob: 0.47


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 50/50 [00:42<00:00,  1.18it/s]

Prob. max counter: 18 | Prob: 0.40


### MAPOCAM

In [11]:
model_shap.use_predict_max = False # to not use max prediction
# little fix to action set
action_set_ = copy.deepcopy(action_set)
for feat in action_set_:
    feat.flip_direction = 1
    feat.update_grid()

method = MAPOCAM(
    action_set_,
    model_shap,
    criteria = "percentile",
    max_changes=3
)

run_experiments(
    method,
    individuals=individuals,
    model=model_shap,
    output_file=f"../results/mlp/german/mapocam_percentile.csv"
)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 50/50 [01:14<00:00,  1.49s/it]


### DICE

In [ ]:
method = Dice(
    X_train,
    Y_train,
    model_shap.clf,
    n_cfs = 1,
    mutable_features = mutable_features,
)

run_experiments(
    method,
    individuals = individuals,
    model = model_shap,
    output_file=f"../results/mlp/german/dice_1sol.csv"
)

### NICE

In [ ]:
method = Nice(
    X_train,
    Y_train,
    model = model,
    cat_features = categoric_features,
)

run_experiments(
    method,
    individuals = individuals,
    model = model_shap,
    output_file=f"../results/mlp/german/nice.csv"
)

## Taiwan

In [4]:
X_train, Y_train, model, outlier_detection, individuals = get_data_model("taiwan", "MLPClassifier")
#convert bools to int
X_train = X_train.astype(int)
individuals = individuals.astype(int)
individuals = individuals.sample(n = 50, random_state=SEED)

In [5]:
not_mutable_features = ['Single', 'Age_in_25_to_40', 'Married', 'Age_lt_25', 'Age_in_40_to_59', 'Age_geq_60', 'EducationLevel']
mutable_features = [feat for feat in X_train.columns if feat not in not_mutable_features]
continuous_features = ["LIMIT_BAL", "BILL_AMT1", "BILL_AMT2", "BILL_AMT3",
                       "BILL_AMT4", "BILL_AMT5", "BILL_AMT6", "PAY_AMT1",
                       "PAY_AMT2", "PAY_AMT3", "PAY_AMT4", "PAY_AMT5",
                       "PAY_AMT6", "MaxBillAmountOverLast6Months", "MaxPaymentAmountOverLast6Months",
                       "MostRecentBillAmount", "MostRecentPaymentAmount", "MostRecentPaymentAmount", "TotalMonthsOverdue"]
categoric_features = [col for col in X_train.columns if col not in continuous_features]

In [8]:
action_set = ActionSet(X = X_train, default_step_size = 0.1, mutable_features = mutable_features)
deep_pipe_explainer = DeepPipeExplainer(model, X_train.sample(100))
model_shap = GeneralClassifier_Shap(model, outlier_detection, X_train, shap_explainer="custom", threshold = 0.5, explainer = deep_pipe_explainer)
model_shap_simple = GeneralClassifier_Shap(model, outlier_detection, X_train)

PermutationExplainer explainer: 1001it [00:27, 22.92it/s]                                                                                                      


### MAPOFCEM

In [9]:
method = MAPOFCEM(
    action_set = action_set,
    classifier = model_shap,
    compare = "percentile",
    max_changes = 3,
    outlier_percentile = 0.01
)

run_experiments(
    method,
    individuals=individuals,
    model=model_shap,
    output_file=f"../results/mlp/taiwan/mapofcem_percentile.csv"
)

  0%|                                                                                                                                   | 0/50 [00:00<?, ?it/s]

  2%|██▍                                                                                                                        | 1/50 [00:42<34:55, 42.77s/it]

Prob. max counter: 1089 | Prob: 0.31


  4%|████▉                                                                                                                      | 2/50 [00:44<14:42, 18.38s/it]

Prob. max counter: 63 | Prob: 0.48


...

Prob. max counter: 420 | Prob: 0.35


 98%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▌  | 49/50 [20:26<00:15, 15.27s/it]

Prob. max counter: 538 | Prob: 0.41


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 50/50 [20:28<00:00, 24.57s/it]

Prob. max counter: 76 | Prob: 0.49


In [10]:
method = MAPOFCEM(
    action_set = action_set,
    classifier = model_shap_simple,
    compare = "percentile",
    max_changes = 3,
    outlier_percentile=0.01
)

run_experiments(
    method,
    individuals=individuals,
    model=model_shap,
    output_file=f"../results/mlp/taiwan/mapofcem_perm_percentile.csv"
)

  0%|                                                                                                                                   | 0/50 [00:00<?, ?it/s]

  2%|██▍                                                                                                                        | 1/50 [01:10<57:29, 70.40s/it]

Prob. max counter: 1136 | Prob: 0.31


  4%|████▉                                                                                                                      | 2/50 [01:12<24:17, 30.37s/it]

Prob. max counter: 63 | Prob: 0.48


...

Prob. max counter: 428 | Prob: 0.35


 98%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▌  | 49/50 [28:10<00:23, 23.68s/it]

Prob. max counter: 539 | Prob: 0.41


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 50/50 [28:13<00:00, 33.87s/it]

Prob. max counter: 76 | Prob: 0.49


### MAPOCAM

In [20]:
model_shap.use_predict_max = False # to not use max prediction
# little fix to action set
action_set_ = copy.deepcopy(action_set)
for feat in action_set_:
    feat.flip_direction = 1
    feat.update_grid()

method = MAPOCAM(
    action_set_,
    model_shap,
    criteria = "percentile",
    max_changes=3
)

run_experiments(
    method,
    individuals=individuals,
    model=model_shap,
    output_file=f"../results/mlp/taiwan/mapocam_percentile.csv"
)

  0%|                                                                                                                                   | 0/50 [00:00<?, ?it/s]

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 50/50 [26:51<00:00, 32.22s/it]


### DICE

In [ ]:
method = Dice(
    X_train,
    Y_train,
    model_shap.clf,
    n_cfs = 1,
    mutable_features = mutable_features,
)

run_experiments(
    method,
    individuals = individuals,
    model = model_shap,
    output_file=f"../results/mlp/taiwan/dice_1sol.csv"
)

### NICE

In [ ]:
method = Nice(
    X_train,
    Y_train,
    model = model,
    cat_features = categoric_features,
)

run_experiments(
    method,
    individuals = individuals,
    model = model_shap,
    output_file=f"../results/mlp/taiwan/nice.csv"
)